In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'..\CCM', nargout=0)
eng.addpath(r'..\CCM\util', nargout=0)

In [2]:
import sys
sys.path.append('..')

from CCM import framework as methods
from CCM import assessment as assess

import SimpleITK as sitk
import numpy as np
import pandas as pd
import os

def GetPointsFromFireTxt(location, object, frame):
    fixed_file_name = os.path.join(location, object, 'cloud_master.csv')
    moving_file_name = os.path.join(location, object, f'cloud_{frame}.csv')
    
    fixed_data = pd.read_csv(fixed_file_name)
    fixed_points = fixed_data.values[:,1:-1].astype(np.float64)

    moving_data = pd.read_csv(moving_file_name)
    moving_points = moving_data.values[:,1:-1].astype(np.float64)
    
    visible_index = np.where(moving_points[:,0]>-1)[0]
    fixed_points = fixed_points[visible_index,:]
    moving_points = moving_points[visible_index,:]
    return [fixed_points, moving_points]

data_location = '../Data/OK/'
# data_objects = {'objects':['Bag1', 'Blanket1', 'Can1', 'Shirt1', 'Shirt2', 'Shirt3'], 'frames':[4, 15, 6, 14, 18, 17]} # for all objects in OK
data_objects = {'objects':['Blanket1'], 'frames':[15]}
               
for object_index in range(len(data_objects['objects'])):
    object = data_objects['objects'][object_index]
    # for frame_index in range(data_objects['frames'][object_index]): # for all frames
    for frame_index in range(1):
        frame = frame_index + 1
        
        print(f'OK dataset, Object: {object}, Frame: {frame}')

        fixed = sitk.ReadImage(os.path.join(data_location, object, 'cloud_master-rgb.png'), sitk.sitkUInt8)
        moving = sitk.ReadImage(os.path.join(data_location, object, f'cloud_{frame}-rgb.png'), sitk.sitkUInt8)
        fixed_points, moving_points = GetPointsFromFireTxt(data_location, object, frame)

        output_path = os.path.join(data_location, 'Results', object, f'frame_{frame}')
        os.makedirs(output_path, exist_ok=True)

        print('\rCCM ...', end='')
        reg = methods.Registration(fixed=fixed, moving=moving, matlab_eng=eng)
        reg.PerformCCMRegistration(
            nlevel=[1], nmigr=[3], niter=[200], step=1, sigma=16,
            feature_detector=reg.detectorFAST, feature_detector_size=7, feature_detector_angle=150, feature_detector_ratio=[5], feature_detector_num=[1000],
            feature_descriptor=reg.descriptorBRISK, feature_descriptor_radius=[0, 3, 5, 7, 9], feature_descriptor_samplenum=[1, 10, 15, 20, 25], feature_descriptor_dmax=5, feature_descriptor_dmin=8,
            feature_match_ratio=1, feature_match_crosscheck=True,
            route_merge=True, route_check=True, route_stay=0.01, mask_valid=False,
            ccm_k=[50], ccm_e=[0.5], ccm_j=[0.5], ccm_t=[0.5], ccm_z=[0.5])
        reg.OutputRegistrationResult(os.path.join(output_path, f'{object}_{frame}_ccm_'))
        print('\rCCM, done.')
        print(assess.Assessment(registration=reg, fixed_points=fixed_points, moving_points=moving_points).AssessRegistration())
        
        break
            

c:\Users\Administrator\Documents\Envs\crowd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OK dataset, Object: Blanket1, Frame: 1
CCM, done.
{'mse': [58.72562174479167, 66.84191670417896, 307200], 'jac': [0.9087425615308155, 0.26570979327287625, 307200], 'tre': [1.3641298491152853, 0.7340707264640861, 50], 'err': [-1, -1, -1], 'SDlogJ': 0.925150005034945, 'dice': [], 'hdd': [], 'time': [5.699373006820679]}
